In [14]:
'''
This script is used for Sensiticity Analysis using OpenLCA.
Codes are developed by Ao Chen @Zweistein.
Connect to IPC server in OpenLCA software before running. (Port: 8080)
TODO: if you're running this script in your machine for the first time, run `product_system_reader.ipynb` first!
Search for TODO in the notebook to see how to run this notebook as needed. 
'''

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('notebook', {'highlight_selected_word': {
    'highlight_across_all_cells': True,
    'only_cells_in_scroll': False,
    'delay': 500,
    'code_cells_only': True,
}})

import os
import csv
import copy
import json
import olca
import uuid
import math
import logging
import traceback
import numpy as np
import pandas as pd
from datetime import datetime, date

from matplotlib import pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import rcParams
import matplotlib.patches as mpatches
import seaborn as sns

params = {'mathtext.default': 'regular' }

client = olca.Client(8080)
client

In [15]:
def get_all_process(client):
    # Get all process dataset in dataframe
    Process_descriptor = client.get_descriptors(olca.Process)
    Process_list = []
    ID_list = []
    Location_list = []

    for process in Process_descriptor:
        Process_list.append(process.name)
        ID_list.append(process.id)
        Location_list.append(process.location)
    Processes_df = pd.DataFrame(list(zip(Process_list,
                                    ID_list, Location_list)),
                                columns=['name', 'id', 'location'])
    return Processes_df

In [16]:
Processes_df = get_all_process(client)

In [17]:
Processes_df.head()

,name,id,location
0,"heat and power co-generation, biogas, gas engi...",16f38842-59a3-3a40-a49b-f23b20175b0b,CH
1,"heat and power co-generation, biogas, gas engi...",be8cc65b-d612-3dea-84c4-00a43d0922ed,CH
2,"market for land tenure, arable land, measured ...",e3f589d2-dbe7-32f1-accb-19de6b323b5c,BR-GO
3,"fluorescent whitening agent production, distyr...",6e57e177-1a06-36bb-aad8-b79c7bc61105,RoW
4,electricity voltage transformation from medium...,7bba9abd-7e6a-36bc-97f5-93432dcfb42f,TH


In [68]:
def SA(Result_path, process_id="", start_point=0.5, end_point=1.5, step_size=0.05, Impact_method = 'ei -  IPCC 2013 no LT'):
    if not os.path.exists(Result_path):
        os.makedirs(Result_path)
    SA_dict = {}
    field_cache = ['InputFlow', 'GWP100a', "Relative change"]
    if len(process_id) != 0:
        try:
            search_process_list = [Processes_df[Processes_df['id']==process_id].iloc[0]]
        except Exception as e:
            print("Invalid process id!")
    else:
        print("No process id is provided!")

    setup = olca.CalculationSetup()
    setup.calculation_type = olca.CalculationType.SIMPLE_CALCULATION # TODO: Can choose SIMPLE_CALCULATION, CONTRIBUTION_ANALYSIS, UPSTREAM_ANALYSIS, REGIONALIZED_CALCULATION, MONTE_CARLO_SIMULATION
    setup.impact_method = client.find(olca.ImpactMethod, Impact_method)
    setup.amount = 1.0

    try:
        SA_dict = json.load(open("SA_record.json", "r"))
    except FileNotFoundError as e:
        json.dump(SA_dict, open("SA_record.json", "w"))

    for i in range(len(search_process_list)):
        result_list_cache = []
        search_process_ID = search_process_list[i]['id']
        search_process_name = search_process_list[i]['name']
        # cached_path = search_process_name.split(' | ')[0]+'_cached_test.csv'
        
        search_process_location = search_process_list[i]['location']
        cached_path = os.path.abspath(Result_path+"/"+search_process_name.split(' | ')[0]+"_"+search_process_location+'_cached.csv')

        process_key =  search_process_name + '|' + search_process_location
        if process_key not in SA_dict.keys():
            SA_dict[process_key] = {}
        
        test_process = client.get(olca.Process, uid=search_process_ID)
        test_input = [exchange for exchange in test_process.exchanges if exchange.input == True]
        for j in range(len(test_input)):
        # for j in range(2): # for test
            initial_value = copy.deepcopy(test_input[j].amount)
            test_amount_range = test_input[j].amount*np.arange(start_point, end_point+step_size, step_size)
            SArange_key = test_input[j].default_provider.name+' | '+test_input[j].default_provider.location+' | '+str(start_point)+' | '+str(end_point)+' | '+str(step_size)
            if SArange_key not in SA_dict[process_key].keys():
                SA_dict[process_key][SArange_key] = {}
                
            print('input flow: '+test_input[j].default_provider.name+test_input[j].default_provider.location)
            print('initial amount:' + str(initial_value))
            for k in range(len(test_amount_range)):
            # for k in range(2): # for test
                test_input[j].amount = test_amount_range[k]
                print('new amount: '+str(test_amount_range[k]))
                client.update(test_process)
                # if len(SA_dict[process_key][SArange_key]) != len(test_amount_range):
                if test_amount_range[k] not in SA_dict[process_key][SArange_key].keys():
                    try:
                        print('Creating new product system...')
                        product_system = client.create_product_system(search_process_ID, default_providers='prefer', preferred_type='UNIT_PROCESS') #TODO: preferred_type can be changed to "SYSTEM_PROCESS" if needed
                        SA_dict[process_key][SArange_key][test_amount_range[k]] = product_system.id
                        product_system_id = product_system.id
                    except Exception as e:
                        print('Failed to create product system of '+search_process_ID)
                        logging.error(traceback.format_exc())
                        SA_dict[process_key][SArange_key] = {}
                        try:
                            print('Re-Creating new product system...')
                            product_system = client.create_product_system(search_process_ID, default_providers='prefer', preferred_type='UNIT_PROCESS') #TODO: preferred_type can be changed to "SYSTEM_PROCESS" if needed
                            SA_dict[process_key][SArange_key][test_amount_range[k]] = product_system.id
                            product_system_id = product_system.id
                        except Exception as e:
                            print('Failed to create product system of '+search_process_ID)
                            logging.error(traceback.format_exc())
                            
                    SA_file = open("SA_record.json", "w")
                    json.dump(SA_dict, SA_file)
                    SA_file.close()
                    
                else:
                    print('Product systems have been created for this process and this range amount.')
                    product_system_id = SA_dict[process_key][SArange_key][test_amount_range[k]]
                
                try:
                    print('Calculating...')
                    setup.product_system = client.get(olca.ProductSystem, uid=product_system_id)
                    result = client.calculate(setup)
                    # client.excel_export(calc_result, 'calc_result/'+product_system_locations[j]+'_'+'calc_result.xlsx')
                    if result.impact_results is not None:
                        for m in range(len(result.impact_results)):
                            GWP_result = result.impact_results[m]
                            if 'GWP 100a' in GWP_result.impact_category.name:
                                GWP100a_value = GWP_result.value
                        result_list_cache.append([test_input[j].default_provider.name+test_input[j].default_provider.location, GWP100a_value, test_amount_range[k]/initial_value])
                        # result_list.append([GWP100a_value, key])
                        with open(cached_path, 'w', newline="") as f:
                            write = csv.writer(f, delimiter=";")
                            write.writerow(["Process name: ", search_process_name+" | "+search_process_location])
                            write.writerow(field_cache)
                            write.writerows(result_list_cache)
                            print('Results cached.')
                    else:
                        print('No impact results: '+process_key)
                except Exception as e:
                    print('Failed to calculate product system: '+product_system_id)
                    logging.error(traceback.format_exc())
                    
                test_input[j].amount = initial_value
                client.update(test_process)
                print('input amount recovered to initial value.')
                
                print('Remaining number of SA variations: '+str(len(test_amount_range)-k-1))
            
            print('Remaining number of input flows to be varied: '+str(len(test_input)-j-1))

In [69]:
process_id = "54aa798a-0cff-4f8d-8a2b-dc6cb356af3d"
start_point=0.5
end_point=1.5
step_size=0.05
Impact_method = 'ei -  IPCC 2013'
Today = date.today().strftime("%d_%b_%Y") # day, month, year
Result_path = Today+'_SA' # TODO: change result path as you need
SA(Result_path=Result_path, process_id=process_id, start_point=start_point, end_point=end_point, step_size=step_size, Impact_method=Impact_method)


input flow: electricity production, oil | electricity, high voltage | Cutoff, URoW
initial amount:0.00706338597306347
new amount: 0.003531692986531735
Creating new product system...
Calculating...
Results cached.
input amount recovered to initial value.
Remaining number of SA variations: 20
new amount: 0.003884862285184909
Creating new product system...
Calculating...
Results cached.
input amount recovered to initial value.
Remaining number of SA variations: 19
new amount: 0.004238031583838083
Creating new product system...
Calculating...
Results cached.
input amount recovered to initial value.
Remaining number of SA variations: 18
new amount: 0.004591200882491256
Creating new product system...
Calculating...
Results cached.
input amount recovered to initial value.
Remaining number of SA variations: 17
new amount: 0.00494437018114443
Creating new product system...
Calculating...
Results cached.
input amount recovered to initial value.
Remaining number of SA variations: 16
new amount: 0

In [ ]:
"54aa798a-0cff-4f8d-8a2b-dc6cb356af3d"

In [ ]:
Processes_df.iloc[0]

In [ ]:
Processes_df[Processes_df['id']=='4f582f90-d0ae-3552-be43-70c730e82af0'].iloc[0]['name'].split(' | ')[0]

In [65]:
test_process = client.get(olca.Process, uid="54aa798a-0cff-4f8d-8a2b-dc6cb356af3d")

In [ ]:
test_exchange = test_process.exchanges[0]

In [66]:
test_input = [exchange for exchange in test_process.exchanges if exchange.input == True]

In [36]:
test_input[0].default_provider.name

'electricity production, solar thermal parabolic trough, 50 MW | electricity, high voltage | Cutoff, U'

In [61]:
test_input[0].amount = 0.1277944082062294
test_input[1].amount = 0.568810584664069
test_input[2].amount = 0.23235346946587163
test_input[3].amount = 0.00706338597306347
test_input[4].amount = 5.8781711690401E-6
client.update(test_process)

'ok'

In [67]:
for input in test_input:
    print(input.default_provider.name)
    print(input.default_provider.location)
    print(input.amount)

electricity production, oil | electricity, high voltage | Cutoff, U
RoW
0.00706338597306347
electricity production, solar thermal parabolic trough, 50 MW | electricity, high voltage | Cutoff, U
RoW
0.23235346946587163
electricity production, natural gas, conventional power plant | electricity, high voltage | Cutoff, U
RoW
0.1277944082062294
electricity production, natural gas, combined cycle power plant | electricity, high voltage | Cutoff, U
RoW
0.568810584664069
electricity production, wind, <1MW turbine, onshore | electricity, high voltage | Cutoff, U
RoW
5.8781711690401e-06


In [44]:
for input in test_input:
    print(input.default_provider.name)
    print(input.default_provider.location)
    print(input.amount)


electricity production, solar thermal parabolic trough, 50 MW | electricity, high voltage | Cutoff, U
RoW
0.23235346946587163
electricity production, oil | electricity, high voltage | Cutoff, U
RoW
0.00706338597306347
electricity production, natural gas, combined cycle power plant | electricity, high voltage | Cutoff, U
RoW
0.568810584664069
electricity production, wind, <1MW turbine, onshore | electricity, high voltage | Cutoff, U
RoW
5.8781711690401e-06
electricity production, natural gas, conventional power plant | electricity, high voltage | Cutoff, U
RoW
0.1277944082062294


In [ ]:
test_input[2].amount = 2.09125441243551e-06
client.update(test_process)

In [ ]:
initial_value = copy.deepcopy(test_input[0].amount)
test_amount_range = test_input[0].amount*np.arange(0.5, 1.55, 0.05)

In [ ]:
for input in test_input:
    print(input.default_provider.location)

In [ ]:
print('input flow: '+test_input[0].default_provider.name)
print('initial amount:' + str(initial_value))
for i in range(len(test_amount_range)):
    test_input[0].amount = test_amount_range[i]
    print('new amount: '+str(test_amount_range[i]))
    
    

In [ ]:
test_exchange.amount

In [ ]:
test_amount = []

In [ ]:
test_amount.append(test_exchange.amount)

In [ ]:
test_exchange.amount = 0.5

In [ ]:
test_amount.append(test_exchange.amount)

In [ ]:
test_process